Add IdentificationToolbox Package to Julia 

In [ ]:
Pkg.rm("IdentificationToolbox")
Pkg.rm("ControlCore")
Pkg.rm("ControlToolbox")
Pkg.rm("Slicot")
Pkg.clone("http://github.com/KTH-AC/ControlCore.jl")
Pkg.clone("http://github.com/neveritt/ControlToolbox.jl")
Pkg.checkout("ControlToolbox","lsim")
Pkg.clone("http://github.com/neveritt/IdentificationToolbox.jl")
Pkg.clone("http://github.com/aytekinar/Slicot.jl")
Pkg.build("Slicot")
Pkg.add("Polynomials")
Pkg.add("ToeplitzMatrices")
Pkg.add("PyPlot")
Pkg.update()

Load the packages

In [ ]:
using IdentificationToolbox
using ControlCore
using PyPlot

Define the true system

In [ ]:
b = [0.1, 0.3, 0.2]
c = [0.5, 0.08]
d = [0.8, 0.1]
f = [0.5, 0.2, 0.1]
B = [0;b]
C = [1;c]
D = [1;d]
F = [1;f]
Ts = 0.1
nk = 1;

Create some data and save the data in an iddata object

In [ ]:
N = 1000
u = randn(N)
λ = 1
e = sqrt(λ)*randn(N)
y = filt(B,F,u) + filt(C,D,e)
data = iddata(y,u,Ts)

Identify an arx model

In [ ]:
na, nb = 5, 5
n = [na; nb; nk]
arx55 = arx(data,n)


Identify an armax model (initialized by iv4)

In [ ]:
na, nb, nc = 3, 3, 3
n = [na; nb; nc; nk]
armax333 = armax(data,n)

BJ model (initialized at true parameters)

In [ ]:
nb, nc, nd, nf = 3, 2, 2, 3
n = [nb; nc; nd; nf; nk]
bj3223 = pem(data, n, [b;c;d;f], BJ())

BJ model (initialized by MORMSM)

In [ ]:
bj3223morsm = pem(data, n, BJ(), MORSM(version=:H))

compare identified models

In [ ]:
yhat_arx55       = predict(data, arx55)
yhat_armax333    = predict(data, armax333)
yhat_bj3223      = predict(data, bj3223)
yhat_bj3223morsm = predict(data, bj3223morsm)

plot(data.y[950:1000],          label="y",          color="blue")
plot(yhat_arx55[950:1000],      label="arx55",      color="red")
plot(yhat_armax333[950:1000],   label="armax333",   color="yellow")
plot(yhat_bj3223[950:1000],     label="bj3223",     color="purple")
plot(yhat_bj3223morsm[950:1000],label="bj3223morsm",color="black")
legend(loc="upper right", fancybox="true")

inspect model

In [ ]:
fieldnames(bj3223)

In [ ]:
bj3223.info.opt

In [ ]:
poles(bj3223.G)